In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_curve
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Data Loading

In [ ]:
train_bankStatement = pd.read_hdf('../data_hdf/train/train_bankStatement.h5',key='data')
train_behaviors = pd.read_hdf('../data_hdf/train/train_behaviors.h5',key='data')
train_creditBill = pd.read_hdf('../data_hdf/train/train_creditBill.h5',key='data')
train_label = pd.read_hdf('../data_hdf/train/train_label.h5',key='data')
train_profile = pd.read_hdf('../data_hdf/train/train_profile.h5',key='data')

test_bankStatement = pd.read_hdf('../data_hdf/test_A/test_bankStatement_A.h5',key='data')
test_behaviors = pd.read_hdf('../data_hdf/test_A/test_behaviors_A.h5',key='data',parse_dates=['日期'])
test_creditBill = pd.read_hdf('../data_hdf/test_A/test_creditBill_A.h5',key='data')
test_profile = pd.read_hdf('../data_hdf/test_A/test_profile_A.h5',key='data')

In [ ]:
train_profile = train_profile.merge(train_label, on='用户标识', how='left')
data = pd.concat([train_profile, test_profile], axis=0).reset_index(drop=True)

# behavior

In [ ]:
temp = pd.concat([train_behaviors,test_behaviors], axis=0)

In [ ]:
date_set = temp['日期'].unique()

date_map = {}
date_map_month = {}
date_map_day = {}

for d in date_set:
    d_int = [int(y) for y in d.split('-')]
    date_map_month[d] = d_int[0]
    date_map_day[d] = d_int[1]
    date_map[d] = d_int[0] * 100 + d_int[1]

temp['月'] = temp['日期'].map(date_map_month)
temp['日'] = temp['日期'].map(date_map_day)
temp['日期'] = temp['日期'].map(date_map)
temp = temp.sort_values(by=['用户标识','日期'])

In [ ]:
for feat in ['行为类型','子类型1','子类型2']:
    tmp = temp.groupby(['用户标识',feat]).count().reset_index()
    tmp = tmp.pivot(index='用户标识', columns=feat, values='日期').reset_index()
    tmp.columns=['用户标识']+['{}{}_count'.format(feat,i) for i in range(temp[feat].nunique())]
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
for feat in ['子类型1','子类型2']:
    tmp = temp.groupby(['用户标识','星期几'])[feat].nunique().reset_index()
    tmp.columns = ['用户标识','星期几','星期几_{}_nunique'.format(feat)]
    tmp = tmp.pivot(index='用户标识', columns='星期几', values='星期几_{}_nunique'.format(feat)).reset_index()
    tmp.columns = ['用户标识']+['{}_{}_nunique'.format(feat,i) for i in range(7)]
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识','月'])['行为类型'].count().reset_index()
tmp = tmp.pivot(index='用户标识', columns='月', values='行为类型').reset_index()
tmp.columns=['用户标识']+['每月行为类型_{}_count'.format(i) for i in range(1,13)]
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp.groupby(['用户标识'])['子类型1','子类型2'].agg({'std'}).reset_index()
tmp.columns = ['用户标识','子类型1_std','子类型2_std']
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
for feat in ['子类型1','子类型2']:
    tmp = temp.groupby(['用户标识','月'])[feat].nunique().reset_index()
    tmp.columns = ['用户标识','月',f'月_{feat}_nunique']
    tmp = tmp.pivot(index='用户标识', columns='月', values=f'月_{feat}_nunique'').reset_index()
    tmp.columns = ['用户标识']+['{}每月_{}_nunique'.format(feat, i) for i in range(1,13)]
    data = data.merge(tmp, on='用户标识', how='left')

# creditbill

In [ ]:
temp = pd.concat([train_creditBill,test_creditBill], axis=0)

temp['账单时间'] = pd.to_datetime(temp['账单时间戳'].values, unit='s')
temp['账单时间-月'] = temp['账单时间'].dt.month
temp = temp.sort_values(by=['用户标识','账单时间戳'])

In [ ]:
for feat in ['上期账单金额','上期还款金额','本期账单余额','信用卡额度']:
    tmp = temp.groupby(['用户标识'])[feat].agg({f'{feat}_mean':'mean',f'{feat}_max':'max',f'{feat}_std':'std',f'{feat}_min':'min',f'{feat}_median':'median'}).reset_index()
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
for feat_I, feat_II in [('上期还款金额','上期账单金额'),('信用卡额度','本期账单余额')]:
    for tag in ['mean','max','min','std']
    data[f'{feat_I}与{feat_II}{tag}差值'] = data[f'{feat_I}_{tag}']-data[f'{feat_II}_{tag}']

In [ ]:
for feat in ['上期账单金额','上期还款金额','本期账单余额']:
    data[f'{feat}_gap'] = data[f'{feat}_max'] - data[f'{feat}_min']
    data[f'{feat}_变异系数'] = data[f'{feat}_std']/data[f'{feat}_mean']

In [ ]:
tmp = temp.groupby(['用户标识'])['还款状态'].max().reset_index()
tmp.columns = ['用户标识','还款状态_max']
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp.groupby(['用户标识','银行标识'])['还款状态'].count().reset_index()
tmp = tmp.pivot(index='用户标识', columns='银行标识', values='还款状态').reset_index()
tmp.columns = ['用户标识'] + ['用户标识_银行标识_{}'.format(i) for i in range(13)]
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp[(temp['上期还款金额']==0) & (temp['上期账单金额']!=0)]
tmp = tmp.groupby(['用户标识'])['上期还款金额'].count().reset_index()
tmp.columns = ['用户标识','上期没还款次数']
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp[temp['上期账单金额']==0]
tmp = tmp.groupby(['用户标识'])['上期账单金额'].count().reset_index()
tmp.columns = ['用户标识','上期账单金额为零_count']
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp[(temp['信用卡额度']==0) & temp['本期账单余额']!=0]
tmp = tmp.groupby(['用户标识'])['本期账单余额'].count().reset_index()
tmp.columns = ['用户标识','信用卡额度为零账单余额不为零']
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
temp['上期还款金额=上期账单金额'] = (temp['上期账单金额'] == temp['上期还款金额']).astype(int)
tmp = temp.groupby(['用户标识'])['上期还款金额=上期账单金额'].count().reset_index()
tmp.columns = ['用户标识','上期还款金额=上期账单金额_count']
data = data.merge(tmp, on='用户标识', how='left')

temp['上期账单金额-上期还款金额'] = temp['上期账单金额'] - temp['上期还款金额']
tmp = temp.groupby(['用户标识'])['上期账单金额-上期还款金额'].agg({'上期账单金额-上期还款金额_max':'max','上期账单金额-上期还款金额_min':'min','上期账单金额-上期还款金额_std':'std','上期账单金额-上期还款金额_mean':'mean'}).reset_index()
data = data.merge(tmp, on='用户标识', how='left')

temp['上期还款金额>上期账单金额'] = (temp['上期账单金额'] > temp['上期还款金额']).astype(int)
tmp = temp.groupby(['用户标识'])['上期还款金额>上期账单金额'].agg({'count'}).reset_index()
tmp.columns = ['用户标识','上期还款金额>上期账单金额_count']
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识','账单时间-月'])['本期账单余额'].std().reset_index()
tmp.columns = ['用户标识','账单时间-月','每月账单余额_std']
tmp = tmp.pivot(index='用户标识', columns='账单时间-月', values='每月账单余额_std').reset_index()
tmp.columns = ['用户标识'] + ['用户标识_每月账单余额_std_{}'.format(i) for i in range(1,13)]
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
for time in [3,6]:
    tmp = temp[temp['账单时间'].dt.hour <time].groupby(['用户标识'])['账单时间'].count().reset_index()
    tmp.columns=['用户标识',f'账单时间<{time}点_count']
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识'])['账单时间戳'].agg({'账单时间戳_max':'max','账单时间戳_min':'min'}).reset_index()
data = data.merge(tmp, on='用户标识', how='left')

temp['本期账单余额-信用卡额度'] = temp['本期账单余额'] - temp['信用卡额度']

In [ ]:
for feat in ['上期账单金额','上期还款金额','本期账单余额','信用卡额度','上期账单金额-上期还款金额','本期账单余额-信用卡额度']:
    for tag in ['std','max','min','mean']:
    tmp = temp.groupby(['用户标识','银行标识'])[feat].agg({tag}).reset_index()
    tmp = tmp.pivot(index='用户标识', columns='银行标识', values=feat).reset_index()
    tmp.columns = ['用户标识'] + ['银行标识_{}_{}_{}'.format(feat,i,tag) for i in range(13)]
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识'])['信用卡额度'].nunique().reset_index()
tmp.columns = ['用户标识','用户信用卡数']
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp.groupby(['用户标识'])['银行标识'].nunique().reset_index()
tmp.columns = ['用户标识','用户银行卡数']
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
def time_diff_stat(ser):
    ts = ser.values // (3600 * 24)
    ts_diff = np.diff(ts)
    if ts_diff.shape[0] < 2:
        return 0, 0, 0, 0
    else:
        return ts_diff.min(), ts_diff.max(), ts_diff.mean(), ts_diff.std()

temp.sort_values(by=['用户标识', '账单时间戳'], inplace=True)
creditbill_stat_g = temp.groupby(['用户标识'])['账单时间戳'].apply(time_diff_stat).reset_index()
for i, st in enumerate(['min', 'max', 'mean', 'std']):
    creditbill_stat_g['账单时间间隔_' + st] = creditbill_stat_g['账单时间戳'].apply(lambda x: x[i])
del creditbill_stat_g['账单时间戳']

data = data.merge(creditbill_stat_g, on='用户标识', how='left')

In [ ]:
for tag in ['min','max','std','mean']:
    tmp = temp.groupby(['用户标识','银行标识'])['账单时间戳'].agg({tag}).reset_index()
    tmp = tmp.pivot(index='用户标识', columns='银行标识', values='账单时间戳').reset_index()
    tmp.columns = ['用户标识'] + ['用户标识_银行标识_账单时间戳_{}_{}'.format(i,tag) for i in range(13)]
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp1 = data[['用户标识_银行标识_账单时间戳_{}_std'.format(i) for i in range(13)]] 
tmp2 = data[['用户标识_银行标识_账单时间戳_{}_mean'.format(i) for i in range(13)]]
tmp1.columns = ['{}'.format(i) for i in range(13)]
tmp2.columns = ['{}'.format(i) for i in range(13)]
tmp = tmp1/tmp2
tmp.columns = ['用户标识_银行标识_账单时间戳_{}_变异系数'.format(i) for i in range(13)]
data = pd.concat([data, tmp], axis=1)

tmp1 = data[['用户标识_银行标识_账单时间戳_{}_max'.format(i) for i in range(13)]] 
tmp2 = data[['用户标识_银行标识_账单时间戳_{}_min'.format(i) for i in range(13)]]
tmp1.columns = ['{}'.format(i) for i in range(13)]
tmp2.columns = ['{}'.format(i) for i in range(13)]
tmp = tmp1 - tmp2
tmp.columns = ['用户标识_银行标识_账单时间戳_{}_gap'.format(i) for i in range(13)]
data = pd.concat([data, tmp], axis=1)

tmp1 = data[['用户标识_银行标识_账单时间戳_{}_max'.format(i) for i in range(13)]] 
tmp2 = data[['用户标识_银行标识_账单时间戳_{}_min'.format(i) for i in range(13)]]
tmp1.columns = ['{}'.format(i) for i in range(13)]
tmp2.columns = ['{}'.format(i) for i in range(13)]
tmp = (tmp1 + tmp2)/2
tmp.columns = ['用户标识_银行标识_账单时间戳_{}_maxmin/2'.format(i) for i in range(13)]
data = pd.concat([data, tmp], axis=1)

In [ ]:
for tag in ['max','min']:
    tmp = temp.groupby(['用户标识','银行标识'])['账单时间戳'].agg({tag}).reset_index()
    tmp.columns = ['用户标识','银行标识',f'账单时间戳_{tag}']
    temp = temp.merge(tmp, on=['用户标识','银行标识'], how='left')

In [ ]:
for tag in ['max','min']:
    for feat in ['上期账单金额','上期还款金额']:
        tmp = temp[temp['账单时间戳']==temp[f'账单时间戳_{tag}']].groupby(['用户标识','银行标识'])[feat].mean().reset_index()
        tmp = tmp.pivot(index='用户标识', columns='银行标识', values=feat).reset_index()
        tmp.columns = ['用户标识'] + ['用户标识_时间戳{}_{}_{}_mean'.format(tag, feat, i) for i in range(13)]
        data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
creditbill_month = temp.groupby(['用户标识', '银行标识', '月'])['上期还款金额'].sum().reset_index()
creditbill_habit = creditbill_month.groupby(['用户标识', '银行标识'])['上期还款金额'].apply(lambda x: x.values[-1] / x.tail(6).mean()).reset_index()
creditbill_habit = creditbill_habit.pivot('用户标识', '银行标识', '上期还款金额').reset_index()
creditbill_habit.columns = ['用户标识'] + ['银行标识%d_还款习惯1/6' % i for i in range(13)]
data = data.merge(creditbill_habit, on='用户标识', how='left')

# bankStatement

In [ ]:
temp = pd.concat([train_bankStatement,test_bankStatement], axis=0)
temp = temp.sort_values(by=['用户标识','流水时间'])
temp['流水'] = pd.to_datetime(temp['流水时间'].values, unit='s')
temp['流水-月'] = temp['流水'].dt.month

In [ ]:
def time_diff_stat(ser):
    ts = ser.values // (3600 * 24)
    ts_diff = np.diff(ts)
    if ts_diff.shape[0] < 2:
        return 0, 0, 0, 0
    else:
        return ts_diff.min(), ts_diff.max(), ts_diff.mean(), ts_diff.std()

temp.sort_values(by=['用户标识', '流水时间'], inplace=True)
creditbill_stat_g = temp.groupby(['用户标识'])['流水时间'].apply(time_diff_stat).reset_index()
for i, st in enumerate(['min', 'max', 'mean', 'std']):
    creditbill_stat_g['流水时间间隔_' + st] = creditbill_stat_g['流水时间'].apply(lambda x: x[i])
del creditbill_stat_g['流水时间']

data = data.merge(creditbill_stat_g, on='用户标识', how='left')

In [ ]:
for feat in ['交易类型','工资收入标记']:
    tmp = temp.groupby(['用户标识'])[feat].max().reset_index()
    tmp.columns = ['用户标识',f'{feat}_max']
    data = data.merge(tmp, on='用户标识', how='left')
    
for tag in ['mean','min']:
    tmp = temp.groupby(['用户标识','交易类型'])['交易金额'].agg({feat}).reset_index()
    tmp = tmp.pivot(index='用户标识', columns='交易类型', values='交易金额').reset_index()
    tmp.columns = ['用户标识'] + ['用户标识_交易类型_交易金额_{}_{}'.format(tag,i) for i in range(1,3)]
    data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识'])['流水时间'].agg({'流水时间_max':'max','流水时间_min':'min'}).reset_index()
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp.groupby(['用户标识'])['交易金额'].agg({'交易金额_max':'max','交易金额_min':'min','交易金额_mean':'mean','交易金额_std':'std'}).reset_index()
data = data.merge(tmp, on='用户标识', how='left')

data['交易金额_gap'] = data['交易金额_max'] - data['交易金额_min']
data['交易金额_变异系数'] = data['交易金额_std'] / data['交易金额_mean']

In [ ]:
tmp = temp.groupby(['用户标识','工资收入标记'])['交易金额'].mean().reset_index()
tmp = tmp.pivot(index='用户标识', columns='工资收入标记', values='交易金额').reset_index()
tmp.columns = ['用户标识'] + ['用户标识_工资收入标记_交易金额_mean_{}'.format(i) for i in range(1,3)]
data = data.merge(tmp, on='用户标识', how='left')

tmp = temp.groupby(['用户标识','流水-月'])['交易金额'].max().reset_index()
tmp = tmp.pivot(index='用户标识', columns='流水-月', values='交易金额').reset_index()
tmp.columns = ['用户标识'] + ['用户标识_每月_交易金额_std_{}'.format(i) for i in range(1,13)]
data = data.merge(tmp, on='用户标识', how='left')

In [ ]:
tmp = temp.groupby(['用户标识','交易类型'])['流水时间'].max().reset_index()
tmp = tmp.pivot(index='用户标识', columns='交易类型', values='流水时间').reset_index()
tmp.columns = ['用户标识'] + ['用户标识_交易类型_流水时间_max_{}'.format(i) for i in range(1,3)]
data = data.merge(tmp, on='用户标识', how='left')

# ALL

In [ ]:
data['行为类型_count_max'] = data[['行为类型{}_count'.format(i) for i in range(8)]].max(axis=1)
data['行为类型_count_min'] = data[['行为类型{}_count'.format(i) for i in range(8)]].min(axis=1)
data['行为类型_count_std'] = data[['行为类型{}_count'.format(i) for i in range(8)]].std(axis=1)
data['行为类型_count_mean'] = data[['行为类型{}_count'.format(i) for i in range(8)]].mean(axis=1)

data['用户标识_银行标识_max'] = data[['用户标识_银行标识_{}'.format(i) for i in range(13)]].max(axis=1)
data['用户标识_银行标识_min'] = data[['用户标识_银行标识_{}'.format(i) for i in range(13)]].min(axis=1)
data['用户标识_银行标识_mean'] = data[['用户标识_银行标识_{}'.format(i) for i in range(13)]].mean(axis=1)

data['用户标识_星期几_count_max'] = data[['用户标识_星期几_{}_count'.format(i) for i in range(7)]].max(axis=1)
data['用户标识_星期几_count_min'] = data[['用户标识_星期几_{}_count'.format(i) for i in range(7)]].min(axis=1)
data['用户标识_星期几_count_mean'] = data[['用户标识_星期几_{}_count'.format(i) for i in range(7)]].mean(axis=1)

data['子类型1_count_max'] = data[['子类型1{}_count'.format(i) for i in range(111)]].max(axis=1)
data['子类型1_count_min'] = data[['子类型1{}_count'.format(i) for i in range(111)]].min(axis=1)
data['子类型1_count_std'] = data[['子类型1{}_count'.format(i) for i in range(111)]].std(axis=1)
data['子类型1_count_mean'] = data[['子类型1{}_count'.format(i) for i in range(111)]].mean(axis=1)

data['子类型2_count_max'] = data[['子类型2{}_count'.format(i) for i in range(59)]].max(axis=1)
data['子类型2_count_min'] = data[['子类型2{}_count'.format(i) for i in range(59)]].min(axis=1)
data['子类型2_count_std'] = data[['子类型2{}_count'.format(i) for i in range(59)]].std(axis=1)
data['子类型2_count_mean'] = data[['子类型2{}_count'.format(i) for i in range(59)]].mean(axis=1)

data['最大最小时间戳_gap'] = data['账单时间戳_max'] - data['账单时间戳_min']
data['最大最小流水时间_gap'] = data['流水时间_max'] - data['流水时间_min']

for col in ['婚姻状态', '性别', '户口类型', '教育程度', '职业']:
    temp_df = data[[col]]
    fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
    data[col+'_fq_enc'] = data[col].map(fq_encode)

# model

In [ ]:
def ks(labels,preds):
    fpr,tpr,thresholds = roc_curve(y_true=labels, y_score=preds)
    return 'ks', max(tpr-fpr), True

In [ ]:
origin_feat = ['婚姻状态', '性别', '户口类型', '教育程度', '职业']+['婚姻状态_fq_enc', '性别_fq_enc', '户口类型_fq_enc', '教育程度_fq_enc', '职业_fq_enc']

creditBill_feat = ['上期账单金额_mean','上期还款金额_mean','本期账单余额_mean','信用卡额度_mean','上期还款金额与上期账单金额mean差值','信用卡额度与本期账单余额mean差值',
                   '上期账单金额_max','上期还款金额_max','本期账单余额_max','信用卡额度_max','上期还款金额与上期账单金额max差值','信用卡额度与本期账单余额max差值',
                  '上期账单金额_min','上期还款金额_min','本期账单余额_min','信用卡额度_min','上期还款金额与上期账单金额min差值','信用卡额度与本期账单余额min差值',
                   '上期账单金额_median','上期还款金额_median','本期账单余额_median','信用卡额度_median',
                   '上期账单金额_gap','上期还款金额_gap','本期账单余额_gap','上期账单金额_变异系数','上期还款金额_变异系数','本期账单余额_变异系数',
                   '账单时间<3点_count','还款状态_max','上期没还款次数']\
                  +['最大最小时间戳_gap']+ ['用户标识_银行标识_{}'.format(i) for i in range(13)] + ['上期还款金额=上期账单金额_count','用户信用卡数','用户银行卡数']+\
                  ['用户标识_银行标识_max','用户标识_银行标识_min','用户标识_银行标识_mean']+['用户标识_每月账单余额_std_{}'.format(i) for i in range(1,13)]+\
                  ['上期账单金额-上期还款金额_max','上期账单金额-上期还款金额_min','上期账单金额-上期还款金额_std']                  
                
                
behavior_feat = ['行为类型{}_count'.format(i) for i in range(8)] + ['子类型1_std','子类型2_std'] + ['用户标识_星期几_{}_count'.format(i) for i in range(7)]+\
                ['行为类型_count_max','行为类型_count_min','行为类型_count_mean','行为类型_count_std','用户标识_星期几_count_max','用户标识_星期几_count_min','用户标识_星期几_count_mean']+\
                ['子类型2_{}_nunique'.format(i) for i in range(7)]
                              

bankstate_feat = ['最大最小流水时间_gap','工资收入标记_max','交易金额_max','交易金额_min','交易金额_std']

all_feat = ['信用卡额度为零账单余额不为零']

test_feat = ['子类型1{}_count'.format(i) for i in range(111)]+['子类型2{}_count'.format(i) for i in range(59)] +\
            ['子类型1_count_max','子类型1_count_min','子类型1_count_mean','子类型1_count_std',
             '子类型2_count_max','子类型2_count_min','子类型2_count_mean','子类型2_count_std']+\
            ['银行标识_上期账单金额_{}_std'.format(i) for i in range(13)]+\
            ['银行标识_上期还款金额_{}_std'.format(i) for i in range(13)]+\
            ['银行标识_本期账单余额_{}_std'.format(i) for i in range(13)]+\
            ['银行标识_信用卡额度_{}_std'.format(i) for i in range(13)]+\
            ['银行标识_上期账单金额_{}_min'.format(i) for i in range(13)]+\
            ['银行标识_上期还款金额_{}_min'.format(i) for i in range(13)]+\
            ['银行标识_本期账单余额_{}_min'.format(i) for i in range(13)]+\
            ['银行标识_信用卡额度_{}_min'.format(i) for i in range(13)]+\
            ['银行标识_上期账单金额_{}_mean'.format(i) for i in range(13)]+\
            ['银行标识_上期还款金额_{}_mean'.format(i) for i in range(13)]+\
            ['银行标识_本期账单余额_{}_mean'.format(i) for i in range(13)]+\
            ['银行标识_信用卡额度_{}_mean'.format(i) for i in range(13)]
            
test_feat_II = ['每月行为类型_{}_count'.format(i) for i in range(1,13)] +['账单时间<6点_count']+\
               ['账单时间间隔_min', '账单时间间隔_max', '账单时间间隔_mean', '账单时间间隔_std','流水时间间隔_min', '流水时间间隔_max', '流水时间间隔_mean', 
                '流水时间间隔_std']+['用户标识_交易类型_交易金额_mean_{}'.format(i) for i in range(1,3)]+\
               ['用户标识_交易类型_交易金额_min_{}'.format(i) for i in range(1,3)] + ['用户标识_工资收入标记_交易金额_mean_1']
        
test_feat_III = ['用户标识_银行标识_账单时间戳_{}_min'.format(i) for i in range(13)]+['用户标识_银行标识_账单时间戳_{}_max'.format(i) for i in range(13)]+\
                ['用户标识_银行标识_账单时间戳_{}_std'.format(i) for i in range(13)]+['用户标识_银行标识_账单时间戳_{}_mean'.format(i) for i in range(13)]+\
                ['用户标识_银行标识_账单时间戳_{}_变异系数'.format(i) for i in range(13)]+['用户标识_银行标识_账单时间戳_{}_gap'.format(i) for i in range(13)]+\
                ['用户标识_银行标识_账单时间戳_{}_maxmin/2'.format(i) for i in range(13)] 

                
test_feat_IV = ['用户标识_时间戳min_上期账单金额_{}_mean'.format(i) for i in range(13)]+\
               ['用户标识_时间戳min_上期还款金额_{}_mean'.format(i) for i in range(13)]+\
               ['用户标识_时间戳max_上期账单金额_{}_mean'.format(i) for i in range(13)]+\
               ['用户标识_时间戳max_上期还款金额_{}_mean'.format(i) for i in range(13)]+\
               ['银行标识_上期账单金额-上期还款金额_max_{}'.format(i) for i in range(13)]+\
               ['银行标识_上期账单金额-上期还款金额_min_{}'.format(i) for i in range(13)]+\
               ['银行标识_上期账单金额-上期还款金额_mean_{}'.format(i) for i in range(13)]+\
               ['银行标识_本期账单余额-信用卡额度_{}_std'.format(i) for i in range(13)]+\
               ['银行标识_本期账单余额-信用卡额度_{}_max'.format(i) for i in range(13)]+\
               ['银行标识_本期账单余额-信用卡额度_{}_min'.format(i) for i in range(13)] #0.519  dim 616  0.5023

test_feat_V =  ['用户标识_交易类型_流水时间_max_{}'.format(i) for i in range(1,3)]+\
               ['子类型1每月_{}_nunique'.format(i) for i in range(1,13)]+\
               ['子类型2每月_{}_nunique'.format(i) for i in range(1,13)]+\
               ['银行标识{}_还款习惯1/6'.format(i) for i in range(13)]
            
###############################################################################################
features =  origin_feat  + behavior_feat + creditBill_feat + bankstate_feat + all_feat + test_feat\
           + test_feat_II + test_feat_III + test_feat_IV + test_feat_V

test_idx = data['标签'].isnull()
train_idx = ~test_idx
train_x = data[features][train_idx].reset_index(drop=True)
train_y = data['标签'][train_idx].reset_index(drop=True)
test_x = data[features][test_idx].reset_index(drop=True)
print(len(features))

In [ ]:
preds = np.zeros((test_x.shape[0], 2))
scores = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*' * 30)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]
    eval_set = [(X_valid, y_valid)]
    lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=64, reg_alpha=0, reg_lambda=1.9, max_bin=64, 
                                max_depth=-1, n_estimators=10000, objective='binary', metrics='None', 
                                bagging_fraction=0.8, is_unbalance=False, bagging_freq=5, min_child_samples=80, 
                                feature_fraction=0.8, learning_rate=0.01, random_state=42, n_jobs=8,
                                )
    lgb_model.fit(X_train, y_train, eval_set=eval_set, eval_metric=ks ,verbose=500, early_stopping_rounds=1000)
    pred = lgb_model.predict_proba(X_valid)
    score = lgb_model.best_score_['valid_0']['ks']
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, lgb_model.best_iteration_, score,np.mean(scores))) 
    preds += lgb_model.predict_proba(test_x)

In [ ]:
imp = pd.DataFrame()
imp['fea'] = train_x.columns
imp['imp'] = lgb_model.feature_importances_ 
imp = imp.sort_values('imp',ascending = False)
imp.head(20)

In [ ]:
result = pd.DataFrame()
result['客户号'] = test_profile['用户标识'] 
result['违约概率'] = preds[:, 1]/len(scores)
print(len(result))
result.to_csv('./out/upload.csv', index=False, header=False)